In [19]:
# Bibliotecas
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from datetime import datetime

# Adicionando a coluna ingestion_timestamp e atualizando com a hora atual
with engine_base01.connect() as conn:
    for table in tables_base01:
        print(f"⏱️ Inserindo ingestion_timestamp na tabela: {table}")
        try:
            # Coloca o nome da tabela entre aspas duplas
            conn.execute(text(f'ALTER TABLE "{table}" ADD COLUMN ingestion_timestamp TEXT'))
        except Exception as e:
            print(f"⚠️ Coluna já existe ou erro ao adicionar na tabela {table}: {e}")

        # Atualiza o valor em todas as linhas da tabela
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        conn.execute(text(f'UPDATE "{table}" SET ingestion_timestamp = :now'), {'now': now})

⏱️ Inserindo ingestion_timestamp na tabela: 00_despesas_alucar
⏱️ Inserindo ingestion_timestamp na tabela: 00_despesas_consigcar
⏱️ Inserindo ingestion_timestamp na tabela: 00_receita_pagseguro_consigcar
⏱️ Inserindo ingestion_timestamp na tabela: 00_vendas_clientes_alucar
⏱️ Inserindo ingestion_timestamp na tabela: 00_vendas_clientes_alucar_estimativa


In [4]:
# Conexão com os bancos de dados bronze
db_path_base01 = "../database/bronze/00_base1.db"
db_path_base02 = "../database/bronze/00_base2.db"

engine_base01 = create_engine(f"sqlite:///{db_path_base01}")
engine_base02 = create_engine(f"sqlite:///{db_path_base02}")

### Verificação das tabelas na base1 (bronze)

In [6]:
# Listando todas as tabelas na base1
inspector = inspect(engine_base01)
tables_base01 = inspector.get_table_names()
print("Tabelas na base1:")
for table in tables_base01:
    print(f"- {table}")

Tabelas na base1:
- 00_despesas_alucar
- 00_despesas_consigcar
- 00_receita_pagseguro_consigcar
- 00_vendas_clientes_alucar
- 00_vendas_clientes_alucar_estimativa


In [8]:
# Verificando a estrutura de cada tabela na base1
for table in tables_base01:
    print(f"\nEstrutura da tabela {table}:")
    columns = inspector.get_columns(table)
    for column in columns:
        print(f"- {column['name']}: {column['type']}")


Estrutura da tabela 00_despesas_alucar:
- DESPESAS: TEXT
- Mês: TEXT
- Valor: TEXT

Estrutura da tabela 00_despesas_consigcar:
- DESPESAS: TEXT
- Mês: TEXT
- Valor: TEXT

Estrutura da tabela 00_receita_pagseguro_consigcar:
- Data: DATE
- Valor: TEXT

Estrutura da tabela 00_vendas_clientes_alucar:
- Nome_(Alucar): TEXT
- Data: DATE
- Mes: BIGINT
- Ano: BIGINT
- Valor_Receita: TEXT

Estrutura da tabela 00_vendas_clientes_alucar_estimativa:
- Nome_(Alucar): TEXT
- Data: DATE
- Mes: BIGINT
- Ano: BIGINT
- Valor_Receita: TEXT


In [9]:
# Verificando amostra de dados de cada tabela na base1
for table in tables_base01:
    print(f"\nAmostra de dados da tabela {table}:")
    try:
        query = f"SELECT * FROM '{table}' LIMIT 5"
        df = pd.read_sql(query, engine_base01)
        display(df)
    except Exception as e:
        print(f"Erro ao acessar tabela {table}: {str(e)}")


Amostra de dados da tabela 00_despesas_alucar:


,DESPESAS,Mês,Valor
0,Lucas Pereira Costa,janeiro,"R$ 7.325,00"
1,Mariana Oliveira Souza,janeiro,"R$ 1.892,41"
2,João Henrique Lima,janeiro,"R$ 2.600,22"
3,Ana Paula Santos,janeiro,"R$ 1.756,23"
4,Aluguel de escritório,janeiro,"R$ 1.800,00"



Amostra de dados da tabela 00_despesas_consigcar:


,DESPESAS,Mês,Valor
0,Ricardo Augusto da Silva Pereira,janeiro,"R$ 3.482,76"
1,Juliana Costa Almeida Lima,janeiro,"R$ 6.215,43"
2,Gabriel Henrique Oliveira Souza,janeiro,"R$ 4.823,36"
3,Larissa Beatriz Santos Ribeiro,janeiro,"R$ 4.890,22"
4,Felipe Augusto Souza Lima,janeiro,"R$ 2.658,90"



Amostra de dados da tabela 00_receita_pagseguro_consigcar:


,Data,Valor
0,2025-01-02,"35.918,27"
1,2025-02-01,"51.472,94"
2,2025-03-01,"34.994,22"
3,2025-04-01,"48.182,77"



Amostra de dados da tabela 00_vendas_clientes_alucar:


,Nome_(Alucar),Data,Mes,Ano,Valor_Receita
0,João Carlos Silva Pereira,2025-01-02,1,2025,"R$ 1.432,00"
1,Maria Fernanda Souza Oliveira,2025-01-03,1,2025,"R$ 4.189,00"
2,Marcos Vinícius Silva Costa,2025-01-06,1,2025,"R$ 118,00"
3,Ana Beatriz Almeida Lima,2025-01-07,1,2025,"R$ 2.756,00"
4,Luiz Felipe Rocha Santos,2025-01-09,1,2025,"R$ 4.631,00"



Amostra de dados da tabela 00_vendas_clientes_alucar_estimativa:


,Nome_(Alucar),Data,Mes,Ano,Valor_Receita
0,Estimativa,2025-06-01,6,2025,"R$ 36.000,00"
1,Estimativa,2025-07-01,7,2025,"R$ 36.000,00"
2,Estimativa,2025-08-01,8,2025,"R$ 36.000,00"
3,Estimativa,2025-09-01,9,2025,"R$ 36.000,00"
4,Estimativa,2025-10-01,10,2025,"R$ 36.000,00"


In [10]:
# Verificando tipos SQL das colunas da tabela 00_vendas_clientes_alucar
print("\nTipos SQL das colunas da tabela 00_vendas_clientes_alucar:")
query = "SELECT * FROM [00_vendas_clientes_alucar] LIMIT 1"
df = pd.read_sql(query, engine_base01)
for column in df.columns:
    column_info = inspector.get_columns('00_vendas_clientes_alucar')
    for col in column_info:
        if col['name'] == column:
            print(f"- {column}: {col['type']}")



Tipos SQL das colunas da tabela 00_vendas_clientes_alucar:
- Nome_(Alucar): TEXT
- Data: DATE
- Mes: BIGINT
- Ano: BIGINT
- Valor_Receita: TEXT


### Verificação das tabelas na base2 (bronze)

In [11]:
# Listando todas as tabelas na base2
inspector = inspect(engine_base02)
tables_base02 = inspector.get_table_names()
print("Tabelas na base2:")
for table in tables_base02:
    print(f"- {table}")

Tabelas na base2:
- 00_vendas_clientes_consigcar


In [12]:
# Verificando a estrutura de cada tabela na base2
for table in tables_base02:
    print(f"\nEstrutura da tabela {table}:")
    columns = inspector.get_columns(table)
    for column in columns:
        print(f"- {column['name']}: {column['type']}")


Estrutura da tabela 00_vendas_clientes_consigcar:
- Data do Pagamento: DATE
- Tipo Produto: TEXT
- Nome: TEXT
- WhatsApp: TEXT
- Quantidade de vezes: BIGINT
- Valor parcela: FLOAT
- Vendedor: TEXT
- Comentarios: FLOAT


In [13]:
# Verificando amostra de dados de cada tabela na base2
for table in tables_base02:
    print(f"\nAmostra de dados da tabela {table}:")
    query = f"SELECT * FROM '{table}' LIMIT 5"
    df = pd.read_sql(query, engine_base02)
    display(df)


print("\nTipos de dados das colunas:")
query = "SELECT * FROM [00_vendas_clientes_consigcar] LIMIT 1"
df = pd.read_sql(query, engine_base02)
for column in df.columns:
    print(f"- {column}: {df[column].dtype}")


Amostra de dados da tabela 00_vendas_clientes_consigcar:


,Data do Pagamento,Tipo Produto,Nome,WhatsApp,Quantidade de vezes,Valor parcela,Vendedor,Comentarios
0,2024-11-11,Completa,Fernanda Gomes Ibrahim Campos,(11) 99123-4567,18,1834.66,Thiago,None
1,2024-11-22,Completa,Patrícia Zanetti Jardim Campos Souza,(21) 99234-5678,6,641.33,Thiago,None
2,2024-11-18,Completa,Eduardo Yamada Dias Vieira Nogueira,(43) 99345-6789,18,1879.30,Thiago,None
3,2024-10-22,Completa,Thiago Lopes Xavier Queiroz,(51) 99456-7890,18,1831.83,Thiago,None
4,2024-10-20,Completa,Vanessa Yamada Farias Krause Teixeira,(61) 99567-8901,6,645.73,Thiago,None



Tipos de dados das colunas:
- Data do Pagamento: object
- Tipo Produto: object
- Nome: object
- WhatsApp: object
- Quantidade de vezes: int64
- Valor parcela: float64
- Vendedor: object
- Comentarios: object


### Análise de dados específicos

In [14]:
# Contagem de registros em cada tabela da base1
print("Contagem de registros na base1:")
for table in tables_base01:
    query = f"SELECT COUNT(*) as total FROM {table}"
    count = pd.read_sql(query, engine_base01).iloc[0,0]
    print(f"- {table}: {count} registros")

Contagem de registros na base1:


OperationalError: (sqlite3.OperationalError) unrecognized token: "00_despesas_alucar"
[SQL: SELECT COUNT(*) as total FROM 00_despesas_alucar]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
# Contagem de registros em cada tabela da base2
print("Contagem de registros na base2:")
for table in tables_base02:
    query = f"SELECT COUNT(*) as total FROM {table}"
    count = pd.read_sql(query, engine_base02).iloc[0,0]
    print(f"- {table}: {count} registros")

Contagem de registros na base2:


OperationalError: (sqlite3.OperationalError) unrecognized token: "00_vendas_clientes_consigcar"
[SQL: SELECT COUNT(*) as total FROM 00_vendas_clientes_consigcar]
(Background on this error at: https://sqlalche.me/e/20/e3q8)